# Lab2

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from icecream import ic
import random
from tqdm.auto import tqdm
from dataclasses import dataclass
import matplotlib.pyplot as plt
# Configure logging
logging.basicConfig(level=logging.WARNING)


In [ ]:

# Load cities data, modify file name to try different problems.
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])

# Initialize distance matrix
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Show the first few rows of CITIES data
ic(CITIES.head())

# Cycle detection function for edge list
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

def valid(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

def tsp_cost(individual):
    tsp = individual.genome  # Access the genome from the Individual
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

def fitness(individual):
    return -tsp_cost(individual)  # Pass Individual to tsp_cost

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None


### Greedy solution 2 slower
Gives slightly better resluts but more computationally expansive, not feasible for china in a reasonable amount of time.
Calculate distances between all cities and sort the distances in an ordered way, then select for the first time the smaller distance and then tries to add a next segment if don't make the graph cyclic, otherwise go to the next and so on.


In [ ]:

# Create segment list with distances
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]
segments.sort(key=lambda e: e[1])  # Sort segments based on distance

def GreedySolution(starting_segment, segments, CITIES, DIST_MATRIX):
    visited = set()
    edges = set()
    times = {i: 0 for i in range(len(CITIES))}
    segment_index = 0

    if not cyclic({tuple(starting_segment[0])}) and all(times[city] < 2 for city in starting_segment[0]):
        city_a, city_b = starting_segment[0]
        times[city_a] += 1
        times[city_b] += 1
        visited |= starting_segment[0]
        edges |= {tuple(starting_segment[0])}

    while segment_index < len(segments):
        next_shortest = segments[segment_index]
        segment_index += 1

        if not cyclic(edges | {tuple(next_shortest[0])}) and all(times[city] < 2 for city in next_shortest[0]):
            city_a, city_b = next_shortest[0]
            times[city_a] += 1
            times[city_b] += 1
            visited |= next_shortest[0]
            edges |= {tuple(next_shortest[0])}

    cities_once = [CITIES.iloc[i].name for i, count in times.items() if count == 1]
    if len(cities_once) == 2:
        final_edge = (cities_once[0], cities_once[1])
        edges.add(final_edge)
        times[cities_once[0]] += 1
        times[cities_once[1]] += 1
    else:
        print("Errore: impossibile chiudere il ciclo con le città disponibili.")

    start_city = cities_once[0]
    tsp = [start_city]
    neighbors = {i: [] for i in range(len(CITIES))}
    for city_a, city_b in edges:
        neighbors[city_a].append(city_b)
        neighbors[city_b].append(city_a)

    current_city = start_city
    previous_city = None

    while True:
        next_city = next(city for city in neighbors[current_city] if city != previous_city)
        tsp.append(next_city)
        if next_city == start_city:
            break
        previous_city = current_city
        current_city = next_city

    # Wrapping tsp as an Individual to calculate fitness
    ic(fitness(Individual(np.array(tsp))))
    return tsp

### Greedy solution 1
Slower but way more fast, espetially for larger problems.
Start from a random city then add the nearest one in the solution until all cities are reached.

In [ ]:

def GreedyOne(CITIES, DIST_MATRIX):

    city = random.choice(range(len(CITIES)))
    
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    visited[city] = True
    tsp = [int(city)]  
    while not np.all(visited):
        dist[:, city] = np.inf 
        closest = np.argmin(dist[city])  #
        logging.debug(
            f"step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} ({DIST_MATRIX[city, closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(int(city)) 
    

    logging.debug(
        f"step: {CITIES.at[tsp[-1], 'name']} -> {CITIES.at[tsp[0], 'name']} ({DIST_MATRIX[tsp[-1], tsp[0]]:.2f}km)"
    )
    tsp.append(tsp[0])
    

    individual = Individual(np.array(tsp))
    ic(fitness(individual))
    
    return tsp

### Creates population
Use the slower greedy approach for creating a population of solutions, the first solution starts from the first segment, the next one starts from a random segment and then try to add segments in an ordered way, from the shortest, this is done to increase diversity in population.

In [ ]:
def GreedyPopulation(segments, CITIES, DIST_MATRIX, population_size):
    population = [] 
    sorted_segments = segments
    starting_segment = segments[0]

    for i in range(population_size):
        sorted_segments = [s for s in sorted_segments if s != starting_segment]
        solution = GreedySolution(starting_segment, sorted_segments, CITIES, DIST_MATRIX)
        individual = Individual(np.array(solution))  # Wrap solution in Individual
        population.append(individual)  # Add Individual to the population
        starting_segment = random.choice(sorted_segments)

    return population


### Creates population 
Use the faster greedy approach for creating a population of solutions, starts from random cities each time.

In [ ]:
def GreedyPopulationOne(CITIES, DIST_MATRIX, population_size):
    population = []
    
    for i in range(population_size):
        tsp_solution = GreedyOne(CITIES, DIST_MATRIX)
        individual = Individual(np.array(tsp_solution))
        population.append(individual)
    
    return population

### Genetic operators
Use inver over crossover, invertion mutation and tournament selection.

In [ ]:

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(parent1: Individual, parent2: Individual):
    # Convert genome to list for easy manipulation
    child_genome = list(parent1.genome.copy())
    
    # Randomly choose a starting gene from the first parent
    start_gene = random.choice(child_genome[1:-1])  # Avoid first and last for TSP cycle
    
    # Find the index of start_gene in both parents
    index_p1 = child_genome.index(start_gene)
    index_p2 = list(parent2.genome).index(start_gene)
    
    # Attempt to find a neighboring edge in parent2 that we can use
    # Try the next gene in parent2 to form the edge
    if index_p2 < len(parent2.genome) - 2:
        next_gene_p2 = parent2.genome[index_p2 + 1]
    else:
        next_gene_p2 = parent2.genome[1]  # Wrap around if needed for TSP cycle
    
    # Insert next_gene_p2 into the child after start_gene
    child_genome.insert(index_p1 + 1, next_gene_p2)
    
    # Remove duplicates while preserving order
    child_genome = list(dict.fromkeys(child_genome))
    
    # Ensure the TSP cycle is valid by matching the first and last elements
    if child_genome[-1] != child_genome[0]:
        child_genome[-1] = child_genome[0]
    
    # Fill any missing nodes
    missing_nodes = set(parent1.genome) - set(child_genome)
    for node in missing_nodes:
        insertion_point = random.randint(1, len(child_genome) - 2)
        child_genome.insert(insertion_point, node)
    
    # Return as an Individual with genome as numpy array
    return Individual(np.array(child_genome))

def muatation(p: Individual):
    genome = p.genome.copy()
    genome_length = len(genome)
    start, end = sorted(random.sample(range(1, genome_length - 1), 2))
    genome[start:end+1] = genome[start:end+1][::-1]
    genome[-1] = genome[0]
    return Individual(genome)

### Execute the EA!

In [ ]:

POPULATION_SIZE=10

#choose wich algorithm to use, comment the other one
population=(GreedyPopulation(segments, CITIES, DIST_MATRIX, POPULATION_SIZE))
#population=GreedyPopulationOne(CITIES,DIST_MATRIX,POPULATION_SIZE)

for i in population:
    i.fitness = fitness(i)

OFFSPRING_SIZE = 4
#generations used for problems: 10_000 italy, 100_000 russia, 200_000 us, 500_000 china
MAX_GENERATIONS = 10_000
best_fitness_per_generation = []

for g in tqdm(range(MAX_GENERATIONS)):
    
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .5:
            p = parent_selection(population)
            o = muatation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    best_fitness_per_generation.append(population[0].fitness)

print("Best Fitness:", population[0].fitness)
print("best path:", population[0].genome)
plt.plot(best_fitness_per_generation)
plt.xlabel("Generations")
plt.ylabel("Best Fitness")
plt.title("Best fitness in population")
plt.show()